In [2]:
!pip install ftfy regex tqdm

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 53.1/53.1 kB 393.2 kB/s eta 0:00:00
     ------------------------------------ 262.1/262.1 kB 848.0 kB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
!pip install torchvision

In [4]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\moham\appdata\local\temp\pip-req-build-f26m_rpp
  Resolved https://github.com/openai/CLIP.git to commit b46f5ac7587d2e1862f8b7b1573179d80dcdd620
  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369454 sha256=bc0a63394abbfee2ff143c056673a6df2b0d3b9e7f0ceaea57c42d27068972d0
  Stored in directory: C:\Users\moham\AppData\Local\Temp\pip-ephem-wheel-cache-__o_ekhx\wheels\c8\e4\e1\11374c111387672fc2068dfbe0d4b424cb9cdd1b2e184a71b5
Successfully built clip


  Running command git clone -q https://github.com/openai/CLIP.git 'C:\Users\moham\AppData\Local\Temp\pip-req-build-f26m_rpp'


In [43]:
import clip
import torch
from PIL import Image

# Load the open CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Function that computes the feature vectors for a batch of images
def compute_clip_features(photos_batch):
    # Load all the photos from the files
    photos = [Image.open(photo_file) for photo_file in photos_batch]
    
    # Preprocess all photos
    photos_preprocessed = torch.stack([preprocess(photo) for photo in photos]).to(device)

    with torch.no_grad():
        # Encode the photos batch to compute the feature vectors and normalize them
        photos_features = model.encode_image(photos_preprocessed)
        photos_features /= photos_features.norm(dim=-1, keepdim=True)

    # Transfer the feature vectors back to the CPU and convert to numpy
    return photos_features.cpu().numpy()

In [47]:
from pathlib import Path
import glob
photos_path = Path("images") 

photos_files = list(photos_path.glob("*.jpg"))

print(f"Photos found: {len(photos_files)}")

Photos found: 311


In [50]:
import math
import numpy as np
import pandas as pd

# Define the batch size so that it fits on your GPU. You can also do the processing on the CPU, but it will be slower.
batch_size = 16

# Path where the feature vectors will be stored
features_path = Path("features")

# Compute how many batches are needed
batches = math.ceil(len(photos_files) / batch_size)

# Process each batch
for i in range(batches):
    print(f"Processing batch {i+1}/{batches}")

   
    batch_features_path = features_path / f"{i:010d}.npy"
    
    # Only do the processing if the batch wasn't processed yet
    if not batch_features_path.exists():
        try:
            # Select the photos for the current batch
            batch_files = photos_files[i*batch_size : (i+1)*batch_size]

            # Compute the features and save to a numpy file
            batch_features = compute_clip_features(batch_files)
            np.save(batch_features_path, batch_features)

        except:
            # Catch problems with the processing to make the process more robust
            print(f'Problem with batch {i}')

Processing batch 1/20
Processing batch 2/20
Processing batch 3/20
Processing batch 4/20
Processing batch 5/20
Processing batch 6/20
Processing batch 7/20
Processing batch 8/20
Processing batch 9/20
Processing batch 10/20
Processing batch 11/20
Processing batch 12/20
Processing batch 13/20
Processing batch 14/20
Processing batch 15/20
Processing batch 16/20
Processing batch 17/20
Processing batch 18/20
Processing batch 19/20
Processing batch 20/20


In [63]:
import numpy as np

# Load all numpy files
features_list = [np.load(features_file) for features_file in sorted(features_path.glob("*.npy"))]

# Concatenate the features and store in a merged file
features = np.concatenate(features_list)
np.save(features_path / "features.npy", features)

In [76]:
features_list = [feature for feature in features]

In [78]:
df = pd.read_csv('dataset.csv')
df['embedding_ViT-B/32'] = features_list
df.to_csv('dataset.csv')

In [79]:
df.head()

,Unnamed: 0,id,path,height,width,depth,description,embedding
0,0,cdc6462c5fa6ce730e46a7c406859415.jpg,c:\Users\moham\Image-Based_Semantic_Search\Ima...,353,235,3,This refreshingly sweet watermelon lemonade is...,"[-0.0531911, 0.038767003, 0.015213832, 0.04112..."
1,1,537f7a51a396e79585c1fcce593a1a80.jpg,c:\Users\moham\Image-Based_Semantic_Search\Ima...,354,236,3,Recipe: Sparkling Strawberry Cucumber Mocktail,"[0.012277477, 0.02332947, -0.018937917, -0.010..."
2,2,72c21d9bedadc9a9f6c70847ab5a1b75--blueberry-mo...,c:\Users\moham\Image-Based_Semantic_Search\Ima...,354,236,3,Blueberries are delicious and CHEAP in the sum...,"[-0.013041987, 0.038087178, -0.018034117, 0.01..."
3,3,31e6e62befb47cbdfa286382503bd19a.jpg,c:\Users\moham\Image-Based_Semantic_Search\Ima...,354,236,3,The Best Virgin Strawberry Daiquiri You’ll Eve...,"[-0.031444285, 0.024132702, 0.013687381, -0.05..."
4,4,f85c2e7e8bb4fcf2e9f6ab83fcd57012.jpg,c:\Users\moham\Image-Based_Semantic_Search\Ima...,471,236,3,Easy Keto Mojito,"[0.012401241, 0.03374445, -0.0057490175, 0.014..."


In [84]:
from numpy import dot
from numpy.linalg import norm
def cosine_similarity(a, b):
    return dot(a, b)/(norm(a)*norm(b))

In [113]:
from PIL import Image
img = Image.open(r"C:\Users\moham\Downloads\soda.jfif")
photo_preprocessed = preprocess(img).unsqueeze(0).to(device)


with torch.no_grad():
    # Encode the photos batch to compute the feature vectors and normalize them
    photo_features = model.encode_image(photo_preprocessed)
    photo_features /= photo_features.norm(dim=-1, keepdim=True)

max = 0, cosine_similarity(features_list[0], photo_features.numpy()[0])
print(max)
for i, feature in enumerate(features_list):
    if cosine_similarity(feature, photo_features.numpy()[0]) > max[1]:
        max = i, cosine_similarity(feature, photo_features.numpy()[0])


(0, 0.3578525)


In [114]:
print(max)

(239, 0.7819175)


In [2]:
Path('images')/df.iloc[239]['id']

NameError: name 'Path' is not defined

In [35]:
d[probs.argmax()]

'BarChick'

In [94]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']